# PyTorch Workflow

Pytorch end-to-end workflow
ref - 
* https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
* https://www.learnpytorch.io/01_pytorch_workflow/
* https://github.com/mrdbourke/pytorch-deep-learning/discussions

In [1]:
what_were_covering = {1: "data (prepare adn load)",
                     2: "build model",
                     3: "fitting model to data (training)",
                     4: "making prediction and evaluating a model (inference)",
                     5: "saving and loading a model",
                     6: "putting it all together"}
what_were_covering

{1: 'data (prepare adn load)',
 2: 'build model',
 3: 'fitting model to data (training)',
 4: 'making prediction and evaluating a model (inference)',
 5: 'saving and loading a model',
 6: 'putting it all together'}

In [2]:
import torch
from torch import nn #nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

#Check Pytorch Version
torch.__version__

'2.0.0'

## Data (preparation and loading)

Data can be almost anything ... in machine learning.

* Excel spreadsheet
* Images of any kind
* Videos
* Audios
* DNA
* Text

Machine Learning's Two Parts:
1. Get data into numerical represntation
2. Build model to learn patterns in numerical representation

To showcase, create some *known* data using linear regression formula.

use a Linear regression formula to make a straight line with *know* **parameters**.

In [3]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1) 
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [4]:
len(X), len(y)

(50, 50)

### Spliting Data into train and test sets (imp concepts)

create training and test sets

In [5]:
# Create train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)